<a href="https://colab.research.google.com/github/akshan-main/next-word/blob/main/wordpredict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

with open('pride_and_prejudice.txt', 'r', encoding='utf-8') as file:
    text = file.read()

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index) + 1

In [ ]:
input_sequences = []
for line in text.split('\n'):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

In [ ]:
max_sequence_len = max([len(seq) for seq in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

In [ ]:
X = input_sequences[:, :-1]
y = input_sequences[:, -1]

In [ ]:
y = np.array(tf.keras.utils.to_categorical(y, num_classes=total_words))

In [ ]:
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model.add(LSTM(150))
model.add(Dense(total_words, activation='softmax'))
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 24, 100)           756100    
                                                                 
 lstm (LSTM)                 (None, 150)               150600    
                                                                 
 dense (Dense)               (None, 7561)              1141711   
                                                                 
Total params: 2048411 (7.81 MB)
Trainable params: 2048411 (7.81 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=100, verbose=1)

Epoch 1/100
3785/3785 [==============================] - 60s 14ms/step - loss: 6.1179 - accuracy: 0.0715
Epoch 2/100
3785/3785 [==============================] - 40s 11ms/step - loss: 5.3499 - accuracy: 0.1292
Epoch 3/100
3785/3785 [==============================] - 40s 10ms/step - loss: 4.9593 - accuracy: 0.1544
Epoch 4/100
3785/3785 [==============================] - 40s 10ms/step - loss: 4.6577 - accuracy: 0.1732
Epoch 5/100
3785/3785 [==============================] - 39s 10ms/step - loss: 4.3909 - accuracy: 0.1893
Epoch 6/100
3785/3785 [==============================] - 39s 10ms/step - loss: 4.1454 - accuracy: 0.2065
Epoch 7/100
3785/3785 [==============================] - 39s 10ms/step - loss: 3.9140 - accuracy: 0.2268
Epoch 8/100
3785/3785 [==============================] - 39s 10ms/step - loss: 3.6950 - accuracy: 0.2508
Epoch 9/100
3785/3785 [==============================] - 38s 10ms/step - loss: 3.4909 - accuracy: 0.2767
Epoch 10/100
3785/3785 [==============================]

In [ ]:
seed_text = "The game"
next_words = 3

for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    predicted = np.argmax(model.predict(token_list), axis=-1)
    output_word = ""
    for word, index in tokenizer.word_index.items():
        if index == predicted:
            output_word = word
            break
    seed_text += " " + output_word

print(seed_text)

1/1 [==============================] - 0s 16ms/step
The game when he was
